# Process Votes
Extract features from votes

## Dependencies / Setup

In [1]:
import pandas as pd
import numpy as np
import pickle
import itertools as iter
from IPython import display as dis

In [2]:
# %load https://gist.github.com/ajp619/ddaa0f35627b066ef528/raw/726005dfa919c7ee0be647f237172029731b7c03/qtutil.py
# silly utility to launch a qtconsole if one doesn't exist

launchConsole = True  # Turn off by setting to False

import psutil

def returnPyIDs():
    pyids = set()
    for pid in psutil.pids():
        try:
            if "python" in psutil.Process(pid).name():
                pyids.add(pid)
        except:
            pass
    return pyids

def launchConsole():
    before_pyids = returnPyIDs()
    %qtconsole
    after_pyids = returnPyIDs()
    newid = after_pyids.difference(before_pyids)
    assert len(newid) == 1
    return list(newid)[0]

try:
    qtid
except NameError:
    if launchConsole:
        qtid = launchConsole()
    
if launchConsole and qtid not in returnPyIDs():
    qtid = launchConsole()
    
qtid

5996

## Load Data

Voting data from 30+ seasons of Survivor was extracted from the site http://survivor.wikia.com/wiki/Main_Page

In [3]:
seasons = pickle.load( open( "wiki_scrape.p", "rb" ) )
seasons.keys()

[u'Palau',
 u'Tocantins',
 u'Borneo',
 u'Africa',
 u'Panama',
 u'Cambodia',
 u'Blood_vs._Water',
 u'Marquesas',
 u'Pearl_Islands',
 u'Vanuatu',
 u'The_Australian_Outback',
 u'Heroes_vs_Villains',
 u'Guatemala',
 u'China',
 u'Worlds_Apart',
 u'Thailand',
 u'The_Amazon',
 u'Cagayan',
 u'South_Pacific',
 u'One_World',
 u'Philippines',
 u'Caramoan',
 u'Gabon',
 u'Micronesia',
 u'Samoa',
 u'All-Stars',
 u'Nicaragua',
 u'Heroes_vs._Villains',
 u'San_Juan_del_Sur',
 u'Cook_Islands',
 u'Redemption_Island',
 u'Fiji']

## Feature extractors

In [4]:
def compare_votes(votes_all):
    # Create a contestant by contestant matrix corresponding to the number of like votes
    
    # Drop columns corresponsing to final counsel (these votes are a different dynamic)
    votes = votes_all.drop(votes_all.columns[votes_all.iloc[0, :] == "Jury Vote"], 1)
    
    like_votes = np.array([None] * votes.shape[0]**2).reshape((votes.shape[0], votes.shape[0]))

    for (i, j) in iter.combinations(range(votes.shape[0]), 2):
        like_votes[i, j] = sum((votes.iloc[i, :] == votes.iloc[j, :]) & #same vote
                               (votes.iloc[i, :] !=  u' \u2014') & #ignore '-' (didn't vote)
                               (votes.iloc[i, :] !=  None) & # ignore 'None' (didn't vote)
                               (votes.iloc[i, :] != '')) #ignore blanks (already eliminated)

    like_votes = pd.DataFrame(like_votes)
    like_votes.index = votes.index
    like_votes.columns = votes.index
    return like_votes

In [5]:
compare_votes(seasons['Borneo']['votes'])

1,Richard,Kelly,Rudy,Susan,Sean,Colleen,Gervase,Jenna,Greg,Gretchen,Joel,Dirk,Ramona,Stacey,B.B.,Sonja
1,,,,,,,,,,,,,,,,
Richard,None,5,9,8,5,0,0,1,0,0,0,1,0,0,0,0
Kelly,None,None,6,6,2,1,0,1,0,0,0,0,0,2,0,1
Rudy,None,None,None,9,6,0,0,1,0,0,0,2,0,0,0,0
Susan,None,None,None,None,6,0,0,1,0,0,0,2,0,0,0,0
Sean,None,None,None,None,None,0,0,1,0,0,0,2,0,0,0,0
Colleen,None,None,None,None,None,None,4,4,2,3,2,0,1,0,0,0
Gervase,None,None,None,None,None,None,None,2,1,1,2,0,2,0,0,0
Jenna,None,None,None,None,None,None,None,None,1,3,2,0,1,0,0,0
Greg,None,None,None,None,None,None,None,None,None,1,0,0,0,0,1,0


####Calculate for all seasons and put into new dict

In [6]:
voteweights = {}
for i in seasons.keys():
    voteweights[i] = compare_votes(seasons[i]['votes'])

In [7]:
for v in voteweights.keys():
    season = seasons[v]
    print "{}: {}".format(v, seasons[v]['url'])
    dis.display(voteweights[v])

Palau: http://survivor.wikia.com/wiki/Survivor:_Palau


1,Tom,Katie,Ian,Jenn,Caryn,Gregg,Stephenie,Janu,Coby,Bobby Jon,Ibrehem,James,Angie,Willard,Kim,Jeff,Ashlee,Jolanda,Wanda,Jonathan
1,,,,,,,,,,,,,,,,,,,,
Tom,None,5,5,5,4,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0
Katie,None,None,6,4,4,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0
Ian,None,None,None,5,4,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0
Jenn,None,None,None,None,3,6,1,1,1,0,0,0,0,2,0,0,0,0,0,0
Caryn,None,None,None,None,None,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0
Gregg,None,None,None,None,None,None,1,1,1,0,0,0,0,2,0,0,0,0,0,0
Stephenie,None,None,None,None,None,None,None,0,0,4,3,5,5,0,3,2,1,0,0,0
Janu,None,None,None,None,None,None,None,None,1,0,0,0,0,0,0,0,0,0,0,0
Coby,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0


Tocantins: http://survivor.wikia.com/wiki/Survivor:_Tocantins


1,J.T.,Stephen,Erinn,Taj,Coach,Debbie,Sierra,Tyson,Brendan,Joe,Sydney,Spencer,Sandy,Jerry,Candace,Carolina
1,,,,,,,,,,,,,,,,
J.T.,None,8,4,5,2,1,1,1,0,3,3,2,1,0,0,0
Stephen,None,None,4,7,2,2,1,0,0,3,3,2,1,0,0,0
Erinn,None,None,None,3,2,2,3,3,2,0,0,0,0,1,0,0
Taj,None,None,None,None,1,1,2,0,0,2,2,1,1,0,0,0
Coach,None,None,None,None,None,5,2,3,2,0,0,0,0,1,0,0
Debbie,None,None,None,None,None,None,2,3,2,0,0,0,0,1,0,0
Sierra,None,None,None,None,None,None,None,2,3,0,0,0,0,1,0,0
Tyson,None,None,None,None,None,None,None,None,2,0,0,0,0,1,0,0
Brendan,None,None,None,None,None,None,None,None,None,0,0,0,0,1,0,0


Borneo: http://survivor.wikia.com/wiki/Survivor:_Borneo


...

1,Yul,Ozzy,Becky,Sundra,Adam,Parvati,Jonathan,Candice,Nate,Jenny,Rebecca,Brad,Jessica,Cristina,Cao Boi,Stephannie,J.P.,Cecilia,Billy,Sekou
1,,,,,,,,,,,,,,,,,,,,
Yul,None,7,9,7,1,1,5,2,0,0,0,0,1,0,1,0,0,0,0,0
Ozzy,None,None,7,9,1,1,4,2,0,0,0,0,0,1,0,0,1,2,0,0
Becky,None,None,None,7,1,1,5,2,0,0,0,0,1,0,1,0,0,0,0,0
Sundra,None,None,None,None,1,1,4,2,0,0,1,0,0,0,0,1,0,1,0,0
Adam,None,None,None,None,None,10,3,5,4,5,4,2,0,2,0,1,0,0,0,0
Parvati,None,None,None,None,None,None,3,5,4,5,4,2,0,2,0,1,0,0,0,0
Jonathan,None,None,None,None,None,None,None,5,2,2,1,0,1,0,1,0,0,0,0,0
Candice,None,None,None,None,None,None,None,None,3,2,1,0,0,0,0,0,0,0,0,0
Nate,None,None,None,None,None,None,None,None,None,4,2,1,0,1,0,0,1,0,0,1


Redemption_Island: http://survivor.wikia.com/wiki/Survivor:_Redemption_Island


1,Rob,Phillip,Natalie,Ashley,Andrea,Mike,Matthew,Grant,Ralph,Steve,Julie,David,Sarita,Stephanie,Krista,Kristina,Russell,Francesca
1,,,,,,,,,,,,,,,,,,
Rob,None,10,10,10,6,0,1,10,0,0,0,0,0,0,0,1,0,0
Phillip,None,None,10,10,8,0,1,8,0,0,0,0,0,0,0,0,0,0
Natalie,None,None,None,12,8,0,0,8,0,0,0,0,0,0,0,0,0,0
Ashley,None,None,None,None,8,0,0,8,0,0,0,0,0,0,0,0,0,0
Andrea,None,None,None,None,None,1,1,6,1,0,0,0,0,0,0,0,0,0
Mike,None,None,None,None,None,None,7,1,8,7,7,7,3,0,0,0,0,0
Matthew,None,None,None,None,None,None,None,2,3,1,1,1,0,1,1,2,1,2
Grant,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,1,0,0
Ralph,None,None,None,None,None,None,None,None,None,8,6,5,3,1,0,0,1,0


Nicaragua: http://survivor.wikia.com/wiki/Survivor:_Nicaragua


1,Fabio,Chase,Sash,Holly,Dan,Jane,Benry,Kelly S.,NaOnka,Brenda,Marty,Alina,Jill,Yve,Kelly B.,Tyrone,Jimmy T.,Jimmy J.,Shannon,Wendy
1,,,,,,,,,,,,,,,,,,,,
Fabio,None,5,6,4,4,2,3,3,2,3,2,1,1,0,0,0,0,0,1,0
Chase,None,None,8,8,5,3,5,3,6,3,1,2,0,0,1,0,0,0,0,0
Sash,None,None,None,6,3,4,3,5,4,5,1,0,1,0,2,0,0,0,0,0
Holly,None,None,None,None,7,6,4,2,5,2,3,2,2,2,0,2,3,1,0,0
Dan,None,None,None,None,None,3,6,1,4,1,5,2,3,3,0,3,2,1,0,0
Jane,None,None,None,None,None,None,1,1,2,1,2,1,2,2,1,2,3,1,0,0
Benry,None,None,None,None,None,None,None,2,5,2,2,2,0,0,1,0,0,0,0,0
Kelly S.,None,None,None,None,None,None,None,None,5,6,1,0,1,0,1,0,0,0,0,0
NaOnka,None,None,None,None,None,None,None,None,None,3,1,2,0,0,1,0,0,0,0,0


##Save info

In [8]:
pickle.dump(voteweights, open( "process_votes.p", "wb" ) )